In [9]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import numpy as np
import sys
import time
import math
import pandas as pd
from sklearn.utils.validation import check_X_y
from sklearn.ensemble import RandomForestClassifier 
from honest_forests.estimators import HonestForestClassifier
from sklearn import tree
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.utils.validation import check_is_fitted
from sklearn.utils.validation import check_X_y
from sklearn.utils.multiclass import check_classification_targets
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance
import scipy.stats as stats
from joblib import Parallel, delayed


In [2]:
def Posterior_After(X,v,X_sample,tree_i):
    
    #Duplicate the sigle sample test
    X_testsample_duplicated = np.tile(X,(X_sample.shape[0],1))
    
    ### Replace test data with values from structure subset   
    X_variable = X_sample[:,v]
    
    X_testsample_duplicated[:,v] = X_variable
    X_testsample_duplicated = X_testsample_duplicated.astype(np.float32)
    
    #Obtain the possible leaves with ignoring variable v
    leaves_indicies = np.unique(tree_i.tree_.apply(X_testsample_duplicated)) 
    value_leaves = np.sum(tree_i.tree_.value[leaves_indicies], axis=0)
    
    #Calculate the posterior probability for sample x
    
    posterior_proba = value_leaves/np.sum(value_leaves) 
    
    
    return value_leaves,posterior_proba
    
    

In [3]:
def SCORE(tree_i,X,y):
    X_structure = X[tree_i.structure_indices_]
    X_honest = X[tree_i.honest_indices_]
    y_honest  = y[tree_i.honest_indices_]
        
    leaves_indicies_before = tree_i.tree_.apply(X_honest.astype(np.float32))
    value_leaves_before = tree_i.tree_.value[leaves_indicies_before]
    classification_before = np.zeros(X_honest.shape[0])
    #error_before = np.zeros((X_honest.shape[0],1))
        
    posterior_proba_before = np.zeros((X_honest.shape[0],2))##n_classes
    value_leaves = np.zeros((X_honest.shape[0],2))
    posterior_proba = np.zeros((X_honest.shape[0],2))
    classification_after = np.zeros(X_honest.shape[0])
    difference_clf = np.zeros(X_honest.shape[0])
    #error_after = np.zeros((X_honest.shape[0],1))
        
    mean_diff = []
    for k in range(0,X_honest.shape[1]):
        print(k)
        for i in range(0,X_honest.shape[0]):
            ##Compute posterior probability before merging
            posterior_proba_before[i,:] = value_leaves_before[i,:]/np.sum(value_leaves_before[i,:])
            ##Compute posterior probability after merging
            value_leaves[i,:],posterior_proba[i,:] = Posterior_After(X_honest[i,:],k,X_structure,tree_i)
            #Compute the difference of posterior possibility for just one class
            
        ## Get the classification_before result
        classification_before = posterior_proba_before.argmax(1)
        #print(classification_before)
        ## Get the classification_after result
        classification_after = posterior_proba.argmax(1)
            
        #Calculate classification error before 
        error_before = np.mean(np.abs(y_honest - classification_before))
        #Calculate classification error after merging leaves
        error_after = np.mean(np.abs(y_honest - classification_after))
        #Calculate the difference of classification error
        difference_clf = error_after - error_before
        mean_diff = np.append(mean_diff,difference_clf)
    return mean_diff

In [4]:
Data = pd.read_csv("GSE99095_normalizedExpression.csv")
Data_gene = Data.iloc[:,0]
Data_GES = Data.iloc[:,1:].T
print(Data_GES)

                0         1         2         3         4         5      \
H2_cell_1   -1.902811  0.012077  0.253047  0.879212 -3.412424  1.545093   
H2_cell_2   -1.826095  0.032311 -2.032669  0.612235 -3.283237  0.673663   
H2_cell_7   -1.767776  0.155079  1.010954  0.897366 -1.175463  0.913850   
H2_cell_8   -1.939080  1.227246  0.684497  1.128768  1.866054  0.776755   
H2_cell_12  -2.467119  0.414571 -0.364441  0.511028  0.497678  0.285301   
...               ...       ...       ...       ...       ...       ...   
P7_cell_218 -2.061681  1.018671 -1.799021  0.966253  0.420283  0.329679   
P7_cell_219 -2.148045 -0.100947  2.272544  0.202707 -0.229254  0.319999   
P7_cell_220 -2.171899 -5.069060 -5.101365 -5.063417 -5.649684 -3.306641   
P7_cell_245 -1.528971  1.255883 -4.458437  2.382207  3.412130  1.966724   
P7_cell_249 -1.568009  2.234874  2.451141  1.692975  1.763232  0.559262   

                6         7         8         9      ...     17248     17249  \
H2_cell_1    2.2200

In [6]:
idx_hp = np.argwhere(np.array(list(map(lambda x: x.find('H'), Data_GES._stat_axis.values.tolist())))==0)
Data_GES['H/P'] = 1
print(idx_hp.ravel().shape)
Data_GES['H/P'][idx_hp.ravel()] = 0
print(Data_GES)

(391,)
                    0         1         2         3         4         5  \
H2_cell_1   -1.902811  0.012077  0.253047  0.879212 -3.412424  1.545093   
H2_cell_2   -1.826095  0.032311 -2.032669  0.612235 -3.283237  0.673663   
H2_cell_7   -1.767776  0.155079  1.010954  0.897366 -1.175463  0.913850   
H2_cell_8   -1.939080  1.227246  0.684497  1.128768  1.866054  0.776755   
H2_cell_12  -2.467119  0.414571 -0.364441  0.511028  0.497678  0.285301   
...               ...       ...       ...       ...       ...       ...   
P7_cell_218 -2.061681  1.018671 -1.799021  0.966253  0.420283  0.329679   
P7_cell_219 -2.148045 -0.100947  2.272544  0.202707 -0.229254  0.319999   
P7_cell_220 -2.171899 -5.069060 -5.101365 -5.063417 -5.649684 -3.306641   
P7_cell_245 -1.528971  1.255883 -4.458437  2.382207  3.412130  1.966724   
P7_cell_249 -1.568009  2.234874  2.451141  1.692975  1.763232  0.559262   

                    6         7         8         9  ...     17249     17250  \
H2_cell_1   

In [7]:
X_train = Data_GES.iloc[:,:-1]
X_train.shape[1]

17258

In [11]:
##Compute feature importance from FIMs
X_train = np.array(Data_GES.iloc[:,:-1])
y_train = np.array(Data_GES.iloc[:,-1])

N = 2
d = X_train.shape[1]
n_estimator = 100

score_hf = np.zeros((N,d))## HFIM
#Calculter Permutation Based HF socre
score_hfp = np.zeros((N,d))## Permutation FIM with honest forest

#Calculter Permutation Based RF socre
score_rfp = np.zeros((N,d))

for num in range(0,N):
    print(num)
    
    

    #Train an Honest Tree
    ###We find that the honest tree nust be full trained？
    ###there will be difference for the rank of feature imporatnces  
    #t_ht = time.time()
    Data = shuffle(Data_GES)
    
    clf = HonestForestClassifier(honest_fraction = 0.8, n_estimators = n_estimator,max_features = d,bootstrap = False)
    clf = clf.fit(X_train, y_train)
    print("finish")
    score_tree = Parallel(n_jobs=-1)(delayed(SCORE)(tree_i,X_train,y_train) for tree_i in (clf.estimators_))

    #Save HF socre
    score_hf[num,:] = np.mean(score_tree,axis = 0)
    
    r_hf = permutation_importance(clf, X_train, y_train,n_repeats=5,random_state=0)
    score_hfp[num,:] = r_hf.importances_mean
    
    clf_rf = RandomForestClassifier(n_estimators = n_estimator,max_features = d)
    clf_rf = clf_rf.fit(X_train, y_train)
    
    r_rf = permutation_importance(clf_rf, X_train, y_train,n_repeats=5,random_state=0,bootstrap = False)
    score_rfp[num,:] = r_rf.importances_mean
    num += 1
    
 

0
finish
